In [ ]:
# 럭키백의 확률

# 데이터 준비하기
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()  # 테이블로 출력해

# input 데이터
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
# target 데이터
fish_target = fish['Species'].to_numpy()

In [ ]:
# 데이터 전처리
# train set, test set 나누기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

# data scaling
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
# K-최근접 이웃의 다중분류 확률 예측
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3) # 기본값은 5, 이번엔 3으로 설정
kn.fit(train_scaled, train_target)

print(kn.classes_)
# class가 어떤식으로 나눠졌지 확인

import numpy as np

# 확률 출력 (predict_proba 메소드 사용)
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))

distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

In [ ]:
# 시그모이드 함수 만들기
import numpy as np
import matplotlib.pyplot as plt

z = np.arange(-5, 5, 0.1)
phi = 1 / (1 + np.exp(-z))

plt.plot(z, phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.show()

In [ ]:
# 로지스틱 회귀로 이진 분류
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt') # 불린 인덱싱
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

from sklearn.linear_model import LogisticRegression

# 객체 만들기
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

# 샘플 5개 (도미, 빙어)
print(lr.predict(train_bream_smelt[:5]))
# 샘플에 대한 확률
print(lr.predict_proba(train_bream_smelt[:5]))

# 가중치
print(lr.coef_, lr.intercept_)

# z값 계산
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

# scipy에서 시그모이드함수 불러오기
from scipy.special import expit
print(expit(decisions))

In [ ]:
# 로지스틱 회귀로 다중 분류
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)

# 정확도 예측
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

# predict_proba 메소드로 확률 출력
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

# 소프트맥스 함수
from scipy.special import softmax

proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))